# TP4 - Non-negative Matrix Factorization
The goal is to study the use of nonnegative matrix factorisation (NMF) for topic extraction from a dataset of text documents. The rationale is to interpret each extracted NMF component as being associated with a specific topic. 

Study and test the following script (introduced  on [scikit](http://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html))

In [1]:
def vectorizeFeatures(_vectorizer=None, _random_state=None):
    # Set default params
    if _vectorizer is None:
        vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=1000, stop_words='english')
    else:
        vectorizer = _vectorizer
    random_state = 1 if _random_state is None else _random_state
    # Fetch data and vectorize
    print("Loading dataset...")
    dataset = fetch_20newsgroups(shuffle=True, random_state=random_state,
                                 remove=('headers', 'footers', 'quotes'))
    data_samples = dataset.data[:2000]        
    t0 = time()
    features = vectorizer.fit_transform(data_samples)
    feature_names = vectorizer.get_feature_names()
    print("done in %0.3fs." % (time() - t0))
    return features, feature_names

In [2]:
def NMFModel(features, _vectorizerName=None, _random_state=None, 
             _beta_loss=None, _init=None, _W=None, _H=None, _K = None):
    
    n_samples = 2000
    n_features = 1000
    n_top_words = 20
    n_components = 10 if _K is None else _K
    vectorizerName = "tf_idf" if _vectorizerName is None else _vectorizerName
    random_state = 1 if _random_state is None else _random_state
    solver = 'cd' if _beta_loss is None else 'mu'
    beta_loss = 'frobenius' if _beta_loss is None else _beta_loss
    init = 'random' if _init is None else _init
    
    print("Fitting the NMF model ("+beta_loss+" norm) with "+vectorizerName+" features, "
          "n_samples=%d and n_features=%d..." % (n_samples, n_features))
    
    t0 = time()
    if _init is None:
        nmf = NMF(n_components=n_components, 
                  random_state=_random_state,
                  solver = solver,
                  beta_loss = beta_loss,
                  alpha=.1, l1_ratio=.5).fit(features)
    else:
        nmf = NMF(n_components=n_components, 
                  random_state=_random_state,
                  solver = solver,
                  beta_loss = beta_loss,
                  alpha=.1, l1_ratio=.5).fit_transform(features, W=_W, H=_H)
    print("done in %0.3fs." % (time() - t0))

    print("\nTopics in NMF model ("+beta_loss+" norm):")
    return nmf, n_top_words

In [3]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [4]:
def runExample(_vectorizer=None, _vectorizerName=None, _random_state=None, _beta_loss=None, 
               _init=None, _W=None, _H=None, _K=None):
    features, feature_names = vectorizeFeatures()
    nmf, n_top_words = NMFModel(features, _vectorizerName, _random_state, _beta_loss, _init, _W, _H, _K)
    print_top_words(nmf, feature_names, n_top_words)

### Q1. Test and comment on the effect of varying the initialisation, especially using random nonnegative values as initial guesses (for W and H coefficients, using the notations introduced during the lecture).

In [5]:
runExample()

NameError: name 'TfidfVectorizer' is not defined

In [ ]:
runExample(_random_state=29)

In [ ]:
runExample(_random_state=69)

### Q2. Compare and comment on the difference between the results obtained with 2 cost compared to the generalised Kullback-Liebler cost.

In [ ]:
runExample(_beta_loss='kullback-leibler')

### Q3. Test and comment on the results obtained using a simpler term-frequency representation as input (as opposed to the TF-IDF representation considered in the code above) when considering the Kullback-Liebler cost.

In [ ]:
_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=1000, stop_words='english')    
runExample(_beta_loss='kullback-leibler', _vectorizer=_vectorizer, _vectorizerName="CountVectorizer")

____________________________________
## Custom NFM Implementation

In [ ]:
###### CUSTOM NMF IMPLEMENTATION ######
# Multiplicative Update Rules for NMF #
# estimation with beta divergences    #
import numpy

# TODO: translate slides 59 [beta-divergence] & 47 [error and special cases]

def custom_NMF(V, K, W=None, H=None, steps=50, beta=0, toll=0.1, show_div=False):
    
    F = len(V) #Number of V rows
    N = len(V[0]) #Number of V columns

    if W is None:
        W = numpy.random.rand(F,K)
        
    if H is None:
        H = numpy.random.rand(K,N)
        
    if N != len(H[0]):
        raise ValueError("Size for H[0] is different - found "+str(len(H[0]))+" in place of "+str(N))
    if F != len(W):
        raise ValueError("Size for F is different - found "+str(len(F))+" in place of "+str(N))
        
    #Setup n_iter
    n_iter = 1
    
    # Setup initial error
    init_error = _beta_div(V,W,H,beta,F,N,K)
    if show_div:
        print("Initial error: "+str(init_error))
    error = init_error
    
    for step in range(steps):
    
#         Tests with whole matrix : multiply = O | dot = *
        upd_UP = numpy.dot(W.T, numpy.multiply(pow(numpy.dot(W,H),beta-2), V))
        upd_DOWN = numpy.dot(W.T, pow(numpy.dot(W,H),beta-1))
        upd = upd_UP / upd_DOWN
        H = numpy.multiply(H, upd)
        
        upd_UP = numpy.dot(numpy.multiply(pow(numpy.dot(W,H),beta-2), V),H.T)
        upd_DOWN = numpy.dot(pow(numpy.dot(W,H),beta-1), H.T)
        upd = upd_UP / upd_DOWN
        W = numpy.multiply(W, upd)
        
        if toll > 0:
            new_error = _beta_div(V,W,H,beta,F,N,K)
            if show_div:
                print("Error on iteration "+str(n_iter)+": " +str(new_error))
            # Check if approximation error relative decrease is below the desired threshold
            if ((error - new_error) / init_error) < toll:
                break
            error = new_error
            
        n_iter += 1
            
    return W, H

def _beta_div(V,W,H,beta,F,N,K):
    div = 0
    # Update beta_divergence
    WH = numpy.dot(W, H)
    for i in range(F):
        for j in range(N):
                x = V[i][j] if V[i][j] != 0 else numpy.finfo(numpy.double).tiny
                y = WH[i][j]
                if beta == 1: # generalized Kullback-Leibler divergence. x log(x/y) - x + y
                    div += x*numpy.log(x/y) - x + y
                elif beta == 0: # Itakura-Saito divergence. (x/y) - log(x/y) -1
                    div += (x/y) * numpy.log(x/y) - 1
                else: # Euclidean distance. (1/beta(beta-1))(x^beta + (beta-1)y^beta - beta*x*y^beta-1)
                    div += 1/(beta*(beta-1))*(pow(x,beta) + (beta-1)*pow(y,beta) - beta*x*pow(y,beta-1))
    return div

#######

In [ ]:
features, feature_names = vectorizeFeatures()

V = numpy.random.rand(features.shape[0], features.shape[1])
V = numpy.array(V) # Data matrix F x N 
K = 2

W, H = custom_NMF(V, K, beta = 1, toll = 0.001, show_div = True)

In [ ]:
runExample(_init='custom', _W=W, _H=H)